# Merging Datasets Together

In [143]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Import custom functions
from custom import removeRegex
from custom import getPatternCount
from sentenceprocess import getLemma
from sentenceprocess import posTag

In [3]:
# Import the initial dataset
df1 = pd.read_csv('../data/raw/labeled_data_raw.csv')

In [5]:
# Print df1
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24783 non-null  int64 
 1   count               24783 non-null  int64 
 2   hate_speech         24783 non-null  int64 
 3   offensive_language  24783 non-null  int64 
 4   neither             24783 non-null  int64 
 5   class               24783 non-null  int64 
 6   tweet               24783 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [7]:
# Rename first column as ID
df1.rename(columns = {'Unnamed: 0': 'id'}, inplace = True)

# Increasing max length for all columns and number of columns
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_info_columns', 500)
pd.set_option('display.max_rows', 500)

In [8]:
# Set ID to index
df1.set_index('id').head(5)

count  hate_speech  offensive_language  neither  class  \
id                                                           
0   3      0            0                   3        2       
1   3      0            3                   0        1       
2   3      0            3                   0        1       
3   3      0            2                   1        1       
4   6      0            6                   0        1       

                                                                                                                                           tweet  
id                                                                                                                                                
0   !!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...  
1   !!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!                                                         
2   !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit                      
3   !!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny                                                                                
4   !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;

In [9]:
df1['class'].value_counts()

1    19190
2    4163 
0    1430 
Name: class, dtype: int64

In [10]:
# Racist or homophobic comments
hateSpeech = df1.loc[df1['class'] == 0]

In [11]:
hateSpeech.head(50)

id  count  hate_speech  offensive_language  neither  class  \
85   85   3      2            1                   0        0       
89   90   3      3            0                   0        0       
110  111  3      3            0                   0        0       
184  186  3      3            0                   0        0       
202  204  3      2            1                   0        0       
204  206  3      2            1                   0        0       
219  221  6      3            2                   1        0       
260  263  3      2            1                   0        0       
312  317  3      2            1                   0        0       
315  320  3      2            1                   0        0       
349  354  3      2            1                   0        0       
352  357  3      2            1                   0        0       
437  443  3      2            1                   0        0       
459  466  3      3            0                   0        0       
519  528  3      2            1                   0        0       
526  535  3      2            1                   0        0       
531  540  3      2            1                   0        0       
540  549  3      3            0                   0        0       
565  574  3      2            1                   0        0       
582  591  3      3            0                   0        0       
583  592  3      2            1                   0        0       
587  597  3      2            1                   0        0       
588  598  3      3            0                   0        0       
603  613  3      2            1                   0        0       
614  624  3      2            1                   0        0       
625  635  3      2            1                   0        0       
635  645  3      2            1                   0        0       
646  656  3      2            1                   0        0       
647  657  3      2            1                   0        0       
663  673  3      2            0                   1        0       
683  693  3      3            0                   0        0       
684  694  3      2            1                   0        0       
690  700  3      2            0                   1        0       
699  710  3      2            0                   1        0       
700  711  3      3            0                   0        0       
731  742  3      2            1                   0        0       
733  744  3      2            1                   0        0       
738  750  3      3            0                   0        0       
759  772  3      2            1                   0        0       
762  775  3      2            1                   0        0       
775  789  3      2            1                   0        0       
776  790  3      2            1                   0        0       
777  791  3      3            0                   0        0       
788  803  3      2            1                   0        0       
792  807  3      2            1                   0        0       
793  808  3      2            1                   0        0       
796  811  3      3            0                   0        0       
797  812  3      2            1                   0        0       
799  815  3      2            1                   0        0       
832  850  3      2            0                   1        0       

                                                                                                                                                                      tweet  
85   "@Blackman38Tide: @WhaleLookyHere @HowdyDowdy11 queer" gaywad                                                                                                           
89   "@CB_Baby24: @white_thunduh alsarabsss" hes a beaner smh you can tell hes a mexican                                                                                     
110  "@DevilGri

In [12]:
# Neither offesnive nor non-offensive speech
neutral = df1.loc[df1['class'] == 2]

In [13]:
neutral.head(50)

id  count  hate_speech  offensive_language  neither  class  \
0    0    3      0            0                   3        2       
40   40   3      0            1                   2        2       
63   63   3      0            0                   3        2       
66   66   3      0            1                   2        2       
67   67   3      0            1                   2        2       
70   70   3      0            0                   3        2       
75   75   3      0            1                   2        2       
115  116  3      0            0                   3        2       
118  119  3      0            0                   3        2       
119  120  3      0            1                   2        2       
120  121  3      0            0                   3        2       
122  123  3      0            0                   3        2       
124  125  3      0            0                   3        2       
140  142  3      0            1                   2        2       
149  151  3      0            1                   2        2       
158  160  3      0            1                   2        2       
164  166  3      0            0                   3        2       
180  182  3      0            0                   3        2       
181  183  3      0            0                   3        2       
186  188  3      0            1                   2        2       
187  189  3      0            1                   2        2       
188  190  3      0            1                   2        2       
190  192  3      0            0                   3        2       
196  198  6      0            0                   6        2       
205  207  3      0            1                   2        2       
216  218  3      0            1                   2        2       
220  222  3      0            1                   2        2       
221  223  3      0            1                   2        2       
240  242  3      0            1                   2        2       
241  243  3      0            1                   2        2       
243  245  3      0            1                   2        2       
247  250  3      0            1                   2        2       
248  251  3      0            1                   2        2       
259  262  3      0            1                   2        2       
269  272  3      0            0                   3        2       
274  277  3      0            0                   3        2       
283  287  3      0            0                   3        2       
313  318  3      0            0                   3        2       
321  326  3      0            1                   2        2       
323  328  3      0            1                   2        2       
329  334  3      0            0                   3        2       
331  336  3      0            0                   3        2       
336  341  3      0            0                   3        2       
341  346  3      0            0                   3        2       
343  348  3      0            1                   2        2       
351  356  3      0            0                   3        2       
356  361  3      0            0                   3        2       
364  369  3      0            0                   3        2       
370  375  3      0            0                   3        2       
372  377  4      0            0                   4        2       

                                                                                                                                                      tweet  
0    !!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...            
40   " momma said no pussy cats inside my doghouse "                                                                                                         
63   "@Addicted2Guys: -SimplyAddictedToGuys http://t.co/1jL4hi8

In [14]:
neutral['count'].value_counts()

3    3919
6    184 
4    37  
9    19  
7    4   
Name: count, dtype: int64

In [15]:
# Offensive but not hate speech
offensiveLang = df1.loc[df1['class'] == 1]

In [16]:
offensiveLang.head(50)

id  count  hate_speech  offensive_language  neither  class  \
1   1   3      0            3                   0        1       
2   2   3      0            3                   0        1       
3   3   3      0            2                   1        1       
4   4   6      0            6                   0        1       
5   5   3      1            2                   0        1       
6   6   3      0            3                   0        1       
7   7   3      0            3                   0        1       
8   8   3      0            3                   0        1       
9   9   3      1            2                   0        1       
10  10  3      0            3                   0        1       
11  11  3      0            3                   0        1       
12  12  3      0            2                   1        1       
13  13  3      0            3                   0        1       
14  14  3      1            2                   0        1       
15  15  3      0            3                   0        1       
16  16  3      0            3                   0        1       
17  17  3      1            2                   0        1       
18  18  3      0            3                   0        1       
19  19  3      0            3                   0        1       
20  20  3      0            3                   0        1       
21  21  3      0            3                   0        1       
22  22  3      0            3                   0        1       
23  23  3      0            3                   0        1       
24  24  3      0            3                   0        1       
25  25  3      0            2                   1        1       
26  26  3      0            3                   0        1       
27  27  3      0            3                   0        1       
28  28  3      0            3                   0        1       
29  29  3      0            3                   0        1       
30  30  3      0            3                   0        1       
31  31  3      0            3                   0        1       
32  32  3      0            3                   0        1       
33  33  3      0            3                   0        1       
34  34  3      0            3                   0        1       
35  35  3      0            3                   0        1       
36  36  3      0            3                   0        1       
37  37  3      0            3                   0        1       
38  38  3      0            2                   1        1       
39  39  3      0            3                   0        1       
41  41  3      0            3                   0        1       
42  42  3      0            3                   0        1       
43  43  3      0            3                   0        1       
44  44  3      0            3                   0        1       
45  45  3      0            3                   0        1       
46  46  3      0            3                   0        1       
47  47  3      0            3                   0        1       
48  48  3      0            3                   0        1       
49  49  3      1            2                   0        1       
50  50  3      1            2                   0        1       
51  51  3      0            3                   0        1       

                                                                                                                                                             tweet  
1   !!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!                                                                           
2   !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit                                        
3   !!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny                                                                            

In [17]:
# Insert new column in our dataframe to account for positive tweets
df1.insert(5, 'positive', 0)

In [18]:
df1['positive'].value_counts()

0    24783
Name: positive, dtype: int64

In [19]:
df1.tail(5)

id  count  hate_speech  offensive_language  neither  positive  \
24778  25291  3      0            2                   1        0          
24779  25292  3      0            1                   2        0          
24780  25294  3      0            3                   0        0          
24781  25295  6      0            6                   0        0          
24782  25296  3      0            0                   3        0          

       class  \
24778  1       
24779  2       
24780  1       
24781  1       
24782  2       

                                                                                                                                                    tweet  
24778  you's a muthaf***in lie &#8220;@LifeAsKing: @20_Pearls @corey_emanuel right! His TL is trash &#8230;. Now, mine? Bible scriptures and hymns&#8221;  
24779  you've gone and broke the wrong heart baby, and drove me redneck crazy                                                                              
24780  young buck wanna eat!!.. dat nigguh like I aint fuckin dis up again                                                                                 
24781  youu got wild bitches tellin you lies                                                                                                               
24782  ~~Ruffled | Ntac Eileen Dahlia - Beautiful color combination of pink, orange, yellow &amp; white. A Coll http://t.co/H0dYEBvnZB

In [24]:
# Import second dataset
df2 = pd.read_csv('../data/raw/train_E6oV3lV.csv')

In [25]:
# Get three columns
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [26]:
# Given that there are 23,353 offensive tweets in the other dataset, select 23,000 positive tweets from df2
dfPos = df2.loc[df2['label'] == 0]

In [27]:
dfPos.head(50)

id  label  \
0   1   0       
1   2   0       
2   3   0       
3   4   0       
4   5   0       
5   6   0       
6   7   0       
7   8   0       
8   9   0       
9   10  0       
10  11  0       
11  12  0       
12  13  0       
15  16  0       
16  17  0       
18  19  0       
19  20  0       
20  21  0       
21  22  0       
22  23  0       
24  25  0       
25  26  0       
26  27  0       
27  28  0       
28  29  0       
29  30  0       
30  31  0       
31  32  0       
32  33  0       
33  34  0       
35  36  0       
36  37  0       
37  38  0       
38  39  0       
39  40  0       
40  41  0       
41  42  0       
42  43  0       
43  44  0       
44  45  0       
45  46  0       
46  47  0       
47  48  0       
48  49  0       
49  50  0       
50  51  0       
51  52  0       
52  53  0       
53  54  0       
54  55  0       

                                                                                                                                              tweet  
0    @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run                                           
1   @user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked                       
2     bihday your majesty                                                                                                                            
3   #model   i love u take with u all the time in urð±!!! ðððð
ð¦ð¦ð¦                                                             
4    factsguide: society now    #motivation                                                                                                          
5   [2/2] huge fan fare and big talking before they leave. chaos and pay disputes when they get there. #allshowandnogo                               
6    @user camping tomorrow @user @user @user @user @user @user @user dannyâ¦                                                                       
7   the next school year is the year for exams.ð¯ can't think about that ð­ #school #exams   #hate #imagine #actorslife #revolutionschool #girl  
8   we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers  â¦                                                           
9    @user @user welcome here !  i'm   it's so #gr8 !                                                                                                
10   â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex                                  
11  we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #            
12  i get to see my daddy today!!   #80days #gettingfed                                                                                              
15  ouch...junior is angryð#got7 #junior #yugyoem   #omg                                                                                          
16  i am thankful for having a paner. #thankful #positive                                                                                            
18  its #friday! ð smiles all around via ig user: @user #cookies make people                                                                      
19  as we all know, essential oils are not made of chemicals.                                                                                        
20  #euro2016 people blaming ha for conceded goal was it fat rooney who gave away free kick knowing bale can hit them from there.                    
21  sad little dude..   #badday #coneofshame #cats #pissed #funny #laughs                                                                            
22  product of the day: happy man #wine tool  who's   it's the #weekend? time to open up &amp; drink up!                                

In [28]:
dfPos23 = dfPos[:23000]

In [29]:
len(dfPos23)

23000

In [30]:
# Setting the index as ID
dfPos23.set_index('id')

label  \
id             
1      0       
2      0       
3      0       
4      0       
5      0       
...   ..       
24725  0       
24726  0       
24727  0       
24728  0       
24729  0       

                                                                                                                                   tweet  
id                                                                                                                                        
1       @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run                             
2      @user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked         
3        bihday your majesty                                                                                                              
4      #model   i love u take with u all the time in urð±!!! ðððð
ð¦ð¦ð¦                                               
5       factsguide: society now    #motivation                                                                                            
...                                                                                       ...                                             
24725  yes, leeds!!   #lesbiunion #girlsweekend                                                                                           
24726  in other non tattoo related news my boy will be #crystalpalace mascot at the home game against liverpool next season !   ð´ðµ  
24727  finally wo agaya     :):)                                                                                                          
24728  ðð ð #love #instagood #photooftheday top.tags #tbt #cute #me #beautiful #followme   #followâ¦                            
24729  with le sister!!!! #family   #igers #instagramhub #instamoment #instatraveling #igdaily #i                                         

[23000 rows x 2 columns]

In [31]:
# Recreate the same structure of the other df
dfPos23.insert(1, 'count', 3)
dfPos23.insert(2, 'hate_speech', 0)
dfPos23.insert(3, 'offensive_language', 0)
dfPos23.insert(4, 'neither', 0)
dfPos23.insert(5, 'positive', 3)

In [32]:
dfPos23

id  count  hate_speech  offensive_language  neither  positive  \
0      1      3      0            0                   0        3          
1      2      3      0            0                   0        3          
2      3      3      0            0                   0        3          
3      4      3      0            0                   0        3          
4      5      3      0            0                   0        3          
...   ..     ..     ..           ..                  ..       ..          
24724  24725  3      0            0                   0        3          
24725  24726  3      0            0                   0        3          
24726  24727  3      0            0                   0        3          
24727  24728  3      0            0                   0        3          
24728  24729  3      0            0                   0        3          

       label  \
0      0       
1      0       
2      0       
3      0       
4      0       
...   ..       
24724  0       
24725  0       
24726  0       
24727  0       
24728  0       

                                                                                                                                   tweet  
0       @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run                             
1      @user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked         
2        bihday your majesty                                                                                                              
3      #model   i love u take with u all the time in urð±!!! ðððð
ð¦ð¦ð¦                                               
4       factsguide: society now    #motivation                                                                                            
...                                        ...                                                                                            
24724  yes, leeds!!   #lesbiunion #girlsweekend                                                                                           
24725  in other non tattoo related news my boy will be #crystalpalace mascot at the home game against liverpool next season !   ð´ðµ  
24726  finally wo agaya     :):)                                                                                                          
24727  ðð ð #love #instagood #photooftheday top.tags #tbt #cute #me #beautiful #followme   #followâ¦                            
24728  with le sister!!!! #family   #igers #instagramhub #instamoment #instatraveling #igdaily #i                                         

[23000 rows x 8 columns]

In [33]:
# Remove existing lable
dfPos23.drop(['label'], axis = 1, inplace = True)

In [34]:
# Insert label as 4 for positive tweets
dfPos23.insert(6, 'class', 3)

In [35]:
dfPos23.head(10)

id  count  hate_speech  offensive_language  neither  positive  class  \
0  1   3      0            0                   0        3         3       
1  2   3      0            0                   0        3         3       
2  3   3      0            0                   0        3         3       
3  4   3      0            0                   0        3         3       
4  5   3      0            0                   0        3         3       
5  6   3      0            0                   0        3         3       
6  7   3      0            0                   0        3         3       
7  8   3      0            0                   0        3         3       
8  9   3      0            0                   0        3         3       
9  10  3      0            0                   0        3         3       

                                                                                                                                             tweet  
0   @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run                                           
1  @user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked                       
2    bihday your majesty                                                                                                                            
3  #model   i love u take with u all the time in urð±!!! ðððð
ð¦ð¦ð¦                                                             
4   factsguide: society now    #motivation                                                                                                          
5  [2/2] huge fan fare and big talking before they leave. chaos and pay disputes when they get there. #allshowandnogo                               
6   @user camping tomorrow @user @user @user @user @user @user @user dannyâ¦                                                                       
7  the next school year is the year for exams.ð¯ can't think about that ð­ #school #exams   #hate #imagine #actorslife #revolutionschool #girl  
8  we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers  â¦                                                           
9   @user @user welcome here !  i'm   it's so #gr8 !

In [36]:
# Checking to see if all IDs are unique
dfPos23['id'] = dfPos23['id'] + 25926

In [37]:
# Final df has 23000 positive tweets
dfPos23.head(10)

id  count  hate_speech  offensive_language  neither  positive  class  \
0  25927  3      0            0                   0        3         3       
1  25928  3      0            0                   0        3         3       
2  25929  3      0            0                   0        3         3       
3  25930  3      0            0                   0        3         3       
4  25931  3      0            0                   0        3         3       
5  25932  3      0            0                   0        3         3       
6  25933  3      0            0                   0        3         3       
7  25934  3      0            0                   0        3         3       
8  25935  3      0            0                   0        3         3       
9  25936  3      0            0                   0        3         3       

                                                                                                                                             tweet  
0   @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run                                           
1  @user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked                       
2    bihday your majesty                                                                                                                            
3  #model   i love u take with u all the time in urð±!!! ðððð
ð¦ð¦ð¦                                                             
4   factsguide: society now    #motivation                                                                                                          
5  [2/2] huge fan fare and big talking before they leave. chaos and pay disputes when they get there. #allshowandnogo                               
6   @user camping tomorrow @user @user @user @user @user @user @user dannyâ¦                                                                       
7  the next school year is the year for exams.ð¯ can't think about that ð­ #school #exams   #hate #imagine #actorslife #revolutionschool #girl  
8  we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers  â¦                                                           
9   @user @user welcome here !  i'm   it's so #gr8 !

In [38]:
df = pd.concat([df1, dfPos23], join = 'inner')

In [39]:
# Final dataset has a good balance between positive and negative tweets
df['positive'].value_counts()

0    24783
3    23000
Name: positive, dtype: int64

In [40]:
# Reshuffle the rows
df = df.sample(frac = 1, random_state = 43).reset_index(drop = True)

In [41]:
len(df)

47783

In [42]:
df['positive'].value_counts()

0    24783
3    23000
Name: positive, dtype: int64

# Data Cleaning

In [46]:
df['tweet_low'] = df['tweet'].apply(lambda x: x if type(x) != str else x.lower())
df

id  count  hate_speech  offensive_language  neither  positive  \
0      28842  3      0            0                   0        3          
1      5684   3      1            2                   0        0          
2      22263  3      1            2                   0        0          
3      40774  3      0            0                   0        3          
4      7082   3      0            0                   3        0          
...     ...  ..     ..           ..                  ..       ..          
47778  26408  3      0            0                   0        3          
47779  18857  3      0            3                   0        0          
47780  43095  3      0            0                   0        3          
47781  20216  3      0            3                   0        0          
47782  14488  3      0            3                   0        0          

       class  \
0      3       
1      1       
2      1       
3      3       
4      2       
...   ..       
47778  3       
47779  1       
47780  3       
47781  1       
47782  1       

                                                                                                                                                              tweet  \
0      13 days to go #gettingthere                                                                                                                                    
1      @anggxo get off my twitter fag                                                                                                                                 
2      These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                                            
3      a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.                                
4      @ramaxe1965 dubya hates todays teabagger movement.                                                                                                             
...                                                   ...                                                                                                             
47778  when quay collab with @user says sold out!!!ð©ð«ð©ð«ð©ð«ð©ð«ð©ð«ð©ð«ð©   #noooo #whyyyy #loveyoudesi #                               
47779  RT @_groovymovie: &#8220;@Shane_A1: Hmu talmbout match but when I pull up its 30 other niccas wit ya &#128530;&#8221; smfh shit like that kill me              
47780  angry squeaking frog  video:     #frog #nature #animals #cute #adorable                                                                                        
47781  RT @obey_jrock__: This is a true ride or die bitch &#128175; http://t.co/y1t8CTQn4U                                                                            
47782  RT @AllHailTaron_: I got the deals for the low. I know you hoes lonely so fuck with these cuffing season specials. &#128184;&#128175; http://t.co/YURpX99Hdb   

                                                                                                                                                          tweet_low  
0      13 days to go #gettingthere                                                                                                                                   
1      @anggxo get off my twitter fag                                                                                                                                
2      these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                                           
3      a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.                               
4      @ramaxe1965 dubya hates todays teabagger movement.                        

In [47]:
len(df)

47783

# Check for duplicates

In [48]:
dup = df[df.duplicated('tweet_low', keep = 'first')]
dup

id  count  hate_speech  offensive_language  neither  positive  \
303    31205  3      0            0                   0        3          
544    50568  3      0            0                   0        3          
645    38535  3      0            0                   0        3          
1027   39543  3      0            0                   0        3          
1085   31253  3      0            0                   0        3          
...      ... ..     ..           ..                  ..       ..          
47718  46972  3      0            0                   0        3          
47728  30566  3      0            0                   0        3          
47758  48509  3      0            0                   0        3          
47772  48055  3      0            0                   0        3          
47780  43095  3      0            0                   0        3          

       class  \
303    3       
544    3       
645    3       
1027   3       
1085   3       
...   ..       
47718  3       
47728  3       
47758  3       
47772  3       
47780  3       

                                                                                                    tweet  \
303    #model   i love u take with u all the time in urð±!!! ðððð
ð¦ð¦ð¦                 
544    #model   i love u take with u all the time in urð±!!! ðððð
ð¦ð¦ð¦                 
645    i finally found a way how to delete old tweets! you might find it useful as well:    #deletetweets   
1027   i finally found a way how to delete old tweets! you might find it useful as well:    #deletetweets   
1085   can #lighttherapy help with   or #depression?   #altwaystoheal #healthy is #happy !!                 
...                                                                                      ...                
47718  @user   bihday rg                                                                                    
47728  get #up get   get  #enjoy #music #today #free #apps #free #music                                     
47758   @user the present moment is filled with joy and happiness @user   #thinkbigsundaywithmarsha         
47772  ð #love #instagood #photooftheday top.tags #tbt #cute #me #beautiful #followme   #followâ¦       
47780  angry squeaking frog  video:     #frog #nature #animals #cute #adorable                              

                                                                                                tweet_low  
303    #model   i love u take with u all the time in urð±!!! ðððð
ð¦ð¦ð¦                
544    #model   i love u take with u all the time in urð±!!! ðððð
ð¦ð¦ð¦                
645    i finally found a way how to delete old tweets! you might find it useful as well:    #deletetweets  
1027   i finally found a way how to delete old tweets! you might find it useful as well:    #deletetweets  
1085   can #lighttherapy help with   or #depression?   #altwaystoheal #healthy is #happy !!                
...                                                                                      ...               
47718  @user   bihday rg                                                                                   
47728  get #up get   get  #enjoy #music #today #free #apps #free #music                                    
47758   @user the present moment is filled with joy and happiness @user   #thinkbigsundaywithmarsha        
47772  ð #love #instagood #photooftheday top.tags #tbt #cute #me #beautiful #followme   #followâ¦      
47780  angry squeaking frog  video:     #frog #nature #animals #cute #adorable                             

[1589 rows x 9 columns]

In [49]:
# Number of duplicates
len(dup)

1589

In [51]:
# Drop duplicates
df = df.drop_duplicates(subset = 'tweet_low', keep = 'first')

In [52]:
len(df)

46194

In [53]:
dup = df[df.duplicated('tweet_low', keep = 'first')]
dup

Empty DataFrame
Columns: [id, count, hate_speech, offensive_language, neither, positive, class, tweet, tweet_low]
Index: []

In [54]:
# Removed duplicates
singleModel = df[df['tweet_low'].str.contains(r'model')]
singleModel

id  count  hate_speech  offensive_language  neither  positive  \
49     41804  3      0            0                   0        3          
368    45453  3      0            0                   0        3          
520    47365  3      0            0                   0        3          
575    33239  3      0            0                   0        3          
1125   28992  3      0            0                   0        3          
1228   41343  3      0            0                   0        3          
2235   44459  3      0            0                   0        3          
2253   33987  3      0            0                   0        3          
3231   8137   3      0            3                   0        0          
3404   41801  3      0            0                   0        3          
4494   34135  3      0            0                   0        3          
5227   12977  3      0            3                   0        0          
5437   45336  3      0            0                   0        3          
5618   22787  3      0            3                   0        0          
5713   28329  3      0            0                   0        3          
6777   13187  3      1            2                   0        0          
6937   46952  3      0            0                   0        3          
7216   49134  3      0            0                   0        3          
7269   34610  3      0            0                   0        3          
7863   18835  3      0            3                   0        0          
8413   31197  3      0            0                   0        3          
8734   35399  3      0            0                   0        3          
8847   17600  3      0            2                   1        0          
9367   33272  3      0            0                   0        3          
9835   30147  3      0            0                   0        3          
10365  33800  3      0            0                   0        3          
10875  45711  3      0            0                   0        3          
10904  26403  3      0            0                   0        3          
12253  49572  3      0            0                   0        3          
12734  26535  3      0            0                   0        3          
12863  27143  3      0            0                   0        3          
14554  26369  3      0            0                   0        3          
14981  16888  6      0            6                   0        0          
15037  33131  3      0            0                   0        3          
15985  59     3      0            3                   0        0          
16545  2160   3      0            3                   0        0          
17135  42244  3      0            0                   0        3          
17637  1428   3      0            3                   0        0          
18096  33428  3      0            0                   0        3          
18279  21354  3      0            3                   0        0          
18339  49538  3      0            0                   0        3          
18834  38193  3      0            0                   0        3          
18952  24622  6      0            6                   0        0          
19313  1427   3      0            3                   0        0          
19378  30557  3      0            0                   0        3          
19766  45114  3      0            0                   0        3          
20202  21044  3      0            2                   1        0          
21170  43397  3      0            0                   0        3          
21181  35462  3      0            0                   0        3          
22001  34747  3      0            0                   0        3          
22139  40036  3      0            0                   0        3          
22247  20372  3      0            3                   0        0          
23388  30430  3      0            0

# Remove retweets

In [58]:
retweet = df[df['tweet_low'].str.contains(r'http://t(?!$)')]
retweet

id  count  hate_speech  offensive_language  neither  positive  \
11     3478   3      1            2                   0        0          
45     18212  3      0            3                   0        0          
56     23582  3      0            0                   3        0          
76     45     3      0            3                   0        0          
83     1194   3      0            0                   3        0          
...     ...  ..     ..           ..                  ..       ..          
47743  16945  3      0            3                   0        0          
47750  8358   3      0            3                   0        0          
47756  25119  4      1            3                   0        0          
47781  20216  3      0            3                   0        0          
47782  14488  3      0            3                   0        0          

       class  \
11     1       
45     1       
56     2       
76     1       
83     2       
...   ..       
47743  1       
47750  1       
47756  1       
47781  1       
47782  1       

                                                                                                                                                              tweet  \
11     @HopOnTheBeast i found her right here, good job unfollowing me btw faggot http://t.co/kD3iSeIoLd                                                               
45     RT @TooSexist: Women complain that chivalry is dead. Yes congratulations bitches, you killed it with feminism http://t.co/Mrc82ZUhOn                           
56     Yankees great Joe DiMaggio reportedly used to beat wife Marilyn Monroe. Here she is in 1954 announcing divorce http://t.co/blS7dalMiF                          
76     " pussy is a powerful drug " &#128517; #HappyHumpDay http://t.co/R8jsymiB5b                                                                                    
83     &#8220;@CaptainYankee2: Two of the best Yankees Derek Jeter and Joe Torre #JoeTorreDay http://t.co/XMyxfDBKOX&#8221; @jordan_luree                             
...                                                                                                                                   ...                             
47743  RT @MyDickNeedsCPR: What lonely hoe made this? http://t.co/eEFThf0tvb                                                                                          
47750  Bored then a hoe! Listening to these fuck ass adults lecturing us with @__vercetti http://t.co/NyBO16RMsh                                                      
47756  trash both ways lol RT @AgdaCoroner: Bitch Killed Herself....Look Like Bill Maher With Makeup on http://t.co/IWLAG2J5Sl                                        
47781  RT @obey_jrock__: This is a true ride or die bitch &#128175; http://t.co/y1t8CTQn4U                                                                            
47782  RT @AllHailTaron_: I got the deals for the low. I know you hoes lonely so fuck with these cuffing season specials. &#128184;&#128175; http://t.co/YURpX99Hdb   

                                                                                                                                                          tweet_low  
11     @hoponthebeast i found her right here, good job unfollowing me btw faggot http://t.co/kd3iseiold                                                              
45     rt @toosexist: women complain that chivalry is dead. yes congratulations bitches, you killed it with feminism http://t.co/mrc82zuhon                          
56     yankees great joe dimaggio reportedly used to beat wife marilyn monroe. here she is in 1954 announcing divorce http://t.co/bls7dalmif                         
76     " pussy is a powerful drug " &#128517; #happyhumpday http://t.co/r8jsymib5b                                                                                   
83     &#8220;@captainyankee2: two of the best yankees derek jeter and joe torre 

In [59]:
retweet['class'].value_counts()

1    1754
2    883 
0    112 
Name: class, dtype: int64

In [60]:
emoji = df[df['tweet'].str.contains(r'#[0-9]')]
emoji['class'].value_counts()

1    4852
2    994 
3    406 
0    213 
Name: class, dtype: int64

In [61]:
posWEmoji = emoji.loc[emoji['class'] == 3]
posWEmoji.head(10)

id  count  hate_speech  offensive_language  neither  positive  class  \
51    48434  3      0            0                   0        3         3       
192   26678  3      0            0                   0        3         3       
430   39963  3      0            0                   0        3         3       
538   32023  3      0            0                   0        3         3       
594   38174  3      0            0                   0        3         3       
610   46640  3      0            0                   0        3         3       
613   25939  3      0            0                   0        3         3       
954   35212  3      0            0                   0        3         3       
1200  30155  3      0            0                   0        3         3       
1226  32523  3      0            0                   0        3         3       

                                                                                                                                                  tweet  \
51     @user just done my first #2minutebeachclean in ages. also got one wet foot putting a starfish back in the water â­ï¸ð  #southwold ð #dâ¦   
192   checked in #holiday     #croatia #13daystogo                                                                                                        
430   "no one is better than dad."   fathersday!!! #fathersday2016 #fatherslove #disney #disneyfana #101dalmatians                                        
538   welcome to candjdays! :) #first #video #vlog #checkitout #youtube #couple #9videos   #phoenix #az #florida                                          
594    @user d-7 opening soon  #miami #restaurant #saltandsugarcafe #20flaglerstreet                                                                      
610    @user pheonix u10s new look team to play noh walkden tomorrow #1stgame                                                                             
613   i get to see my daddy today!!   #80days #gettingfed                                                                                                 
954   popsy &amp; little all ready for @user   #10minutestogo                                                                                             
1200  i'm off to #florida #usa in #july for #3 #weeks woop woop   #holiday                                                                                
1226  the new baby is on her way! xx #700d #cannon                                                                                                        

                                                                                                                                              tweet_low  
51     @user just done my first #2minutebeachclean in ages. also got one wet foot putting a starfish back in the water â­ï¸ð  #southwold ð #dâ¦  
192   checked in #holiday     #croatia #13daystogo                                                                                                       
430   "no one is better than dad."   fathersday!!! #fathersday2016 #fatherslove #disney #disneyfana #101dalmatians                                       
538   welcome to candjdays! :) #first #video #vlog #checkitout #youtube #couple #9videos   #phoenix #az #florida                                         
594    @user d-7 opening soon  #miami #restaurant #saltandsugarcafe #20flaglerstreet                                                                     
610    @user pheonix u10s new look team to play noh walkden tomorrow #1stgame                                                                            
613   i get to see my daddy today!!   #80days #gettingfed                                                                                                
954   popsy &amp; little all ready for @user   #10minutestogo                                                                                            
1200  i'm off to #florida

In [64]:
# Delete URLs
df['no_url'] = np.vectorize(removeRegex)(df['tweet_low'], "https?://[A-za-z0-9./]*")

In [66]:
df.head(15)

id  count  hate_speech  offensive_language  neither  positive  class  \
0   28842  3      0            0                   0        3         3       
1   5684   3      1            2                   0        0         1       
2   22263  3      1            2                   0        0         1       
3   40774  3      0            0                   0        3         3       
4   7082   3      0            0                   3        0         2       
5   33371  3      0            0                   0        3         3       
6   27911  3      0            0                   0        3         3       
7   42015  3      0            0                   0        3         3       
8   6788   3      2            1                   0        0         0       
9   43002  3      0            0                   0        3         3       
10  38422  3      0            0                   0        3         3       
11  3478   3      1            2                   0        0         1       
12  40522  3      0            0                   0        3         3       
13  27249  3      0            0                   0        3         3       
14  44284  3      0            0                   0        3         3       

                                                                                                                                tweet  \
0   13 days to go #gettingthere                                                                                                         
1   @anggxo get off my twitter fag                                                                                                      
2   These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3   a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.     
4   @ramaxe1965 dubya hates todays teabagger movement.                                                                                  
5   i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness?   @user          
6   getting   for this weekends shows! #country #music #lylepierce                                                                      
7    @user my final legislative session day has officially begun!   @user @user #albany                                                 
8   @lucas_wright955 @MichaelGT03 faggots                                                                                               
9   a #bikini kind of life ð´ summer #palmtrees #breeze   #place #cali #california   #swimwearâ¦                                     
10  very sad people with very sad lives, gossiping about people who are fuck all to do with them   #getalife                            
11  @HopOnTheBeast i found her right here, good job unfollowing me btw faggot http://t.co/kD3iSeIoLd                                    
12  missy elliot for marc jacobs is supa dupa fly and i'm   ab it                                                                       
13  working on something ð.                                                                                                          
14  this week like....                                                                                                                  

                                                                                                                            tweet_low  \
0   13 days to go #gettingthere                                                                                                         
1   @anggxo get off my twitter fag                                                                                                      
2   these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3

In [79]:
df['handle_count'] = np.vectorize(getPatternCount)(df['tweet_low'], "@[\w]*")

In [80]:
df['handle_count'].value_counts()

0     23968
1     15967
2     4176 
3     1344 
4     416  
5     158  
6     89   
8     31   
7     26   
9     12   
10    6    
11    1    
Name: handle_count, dtype: int64

In [81]:
df.head(5)

id  count  hate_speech  offensive_language  neither  positive  class  \
0  28842  3      0            0                   0        3         3       
1  5684   3      1            2                   0        0         1       
2  22263  3      1            2                   0        0         1       
3  40774  3      0            0                   0        3         3       
4  7082   3      0            0                   3        0         2       

                                                                                                                               tweet  \
0  13 days to go #gettingthere                                                                                                         
1  @anggxo get off my twitter fag                                                                                                      
2  These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3  a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.     
4  @ramaxe1965 dubya hates todays teabagger movement.                                                                                  

                                                                                                                           tweet_low  \
0  13 days to go #gettingthere                                                                                                         
1  @anggxo get off my twitter fag                                                                                                      
2  these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3  a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.     
4  @ramaxe1965 dubya hates todays teabagger movement.                                                                                  

                                                                                                                              no_url  \
0  13 days to go #gettingthere                                                                                                         
1  @anggxo get off my twitter fag                                                                                                      
2  these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3  a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.     
4  @ramaxe1965 dubya hates todays teabagger movement.                                                                                  

   handle_count  
0  0             
1  1             
2  0             
3  1             
4  1

In [82]:
# Remove Twitter handles
df['no_handle'] = np.vectorize(removeRegex)(df['no_url'], "@[\w]*")

In [83]:
df.tail(10)

id  count  hate_speech  offensive_language  neither  positive  \
47771  30963  3      0            0                   0        3          
47773  42033  3      0            0                   0        3          
47774  40465  3      0            0                   0        3          
47775  16694  3      0            3                   0        0          
47776  29931  3      0            0                   0        3          
47777  5463   3      0            3                   0        0          
47778  26408  3      0            0                   0        3          
47779  18857  3      0            3                   0        0          
47781  20216  3      0            3                   0        0          
47782  14488  3      0            3                   0        0          

       class  \
47771  3       
47773  3       
47774  3       
47775  1       
47776  3       
47777  1       
47778  3       
47779  1       
47781  1       
47782  1       

                                                                                                                                                              tweet  \
47771   @user '' x'mas &amp; my bihday disney ! ''   #love #thanks #karen #â¡                                                                                        
47773  my soul is happiest on the water! #soul   #happier #happiest #water #ocean #beach #caliâ¦                                                                     
47774   â #nzd/usd post-rbnz rally almost reversed, 0.7000 closer   #blog #silver #gold #forex                                                                      
47775  RT @MAKEUP_SEX: trash talked by many . hated by some . & guess how many fucks i give , its less than one .                                                     
47776    #bihday to leo's mom, #celia ..                                                                                                                              
47777  @_ElenaRaquel_ its swag bitch aha                                                                                                                              
47778  when quay collab with @user says sold out!!!ð©ð«ð©ð«ð©ð«ð©ð«ð©ð«ð©ð«ð©   #noooo #whyyyy #loveyoudesi #                               
47779  RT @_groovymovie: &#8220;@Shane_A1: Hmu talmbout match but when I pull up its 30 other niccas wit ya &#128530;&#8221; smfh shit like that kill me              
47781  RT @obey_jrock__: This is a true ride or die bitch &#128175; http://t.co/y1t8CTQn4U                                                                            
47782  RT @AllHailTaron_: I got the deals for the low. I know you hoes lonely so fuck with these cuffing season specials. &#128184;&#128175; http://t.co/YURpX99Hdb   

                                                                                                                                                          tweet_low  \
47771   @user '' x'mas &amp; my bihday disney ! ''   #love #thanks #karen #â¡                                                                                        
47773  my soul is happiest on the water! #soul   #happier #happiest #water #ocean #beach #caliâ¦                                                                     
47774   â #nzd/usd post-rbnz rally almost reversed, 0.7000 closer   #blog #silver #gold #forex                                                                      
47775  rt @makeup_sex: trash talked by many . hated by some . & guess how many fucks i give , its less than one .                                                     
47776    #bihday to leo's mom, #celia ..                                                                                                                              
47777  @_elenaraquel_ its swag bitch aha                                                                                                                              


In [84]:
# Remove special characters (except hashtags and apostrophes). Replace with a whitespace.
df['no_special'] = df['no_handle'].str.replace("[^a-zA-Z#']", " ")

In [85]:
df.head(15)

id  count  hate_speech  offensive_language  neither  positive  class  \
0   28842  3      0            0                   0        3         3       
1   5684   3      1            2                   0        0         1       
2   22263  3      1            2                   0        0         1       
3   40774  3      0            0                   0        3         3       
4   7082   3      0            0                   3        0         2       
5   33371  3      0            0                   0        3         3       
6   27911  3      0            0                   0        3         3       
7   42015  3      0            0                   0        3         3       
8   6788   3      2            1                   0        0         0       
9   43002  3      0            0                   0        3         3       
10  38422  3      0            0                   0        3         3       
11  3478   3      1            2                   0        0         1       
12  40522  3      0            0                   0        3         3       
13  27249  3      0            0                   0        3         3       
14  44284  3      0            0                   0        3         3       

                                                                                                                                tweet  \
0   13 days to go #gettingthere                                                                                                         
1   @anggxo get off my twitter fag                                                                                                      
2   These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3   a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.     
4   @ramaxe1965 dubya hates todays teabagger movement.                                                                                  
5   i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness?   @user          
6   getting   for this weekends shows! #country #music #lylepierce                                                                      
7    @user my final legislative session day has officially begun!   @user @user #albany                                                 
8   @lucas_wright955 @MichaelGT03 faggots                                                                                               
9   a #bikini kind of life ð´ summer #palmtrees #breeze   #place #cali #california   #swimwearâ¦                                     
10  very sad people with very sad lives, gossiping about people who are fuck all to do with them   #getalife                            
11  @HopOnTheBeast i found her right here, good job unfollowing me btw faggot http://t.co/kD3iSeIoLd                                    
12  missy elliot for marc jacobs is supa dupa fly and i'm   ab it                                                                       
13  working on something ð.                                                                                                          
14  this week like....                                                                                                                  

                                                                                                                            tweet_low  \
0   13 days to go #gettingthere                                                                                                         
1   @anggxo get off my twitter fag                                                                                                      
2   these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3

In [86]:
# Remove single hashtags with nothing following them
df['remove_empty_hashtag'] = np.vectorize(removeRegex)(df['no_special'], " # ")

In [87]:
# Counting length of tweets after URLs are removed. Use this to see if there is a correlation between length of a tweet and the sentiment
df['tweet_length'] = df['remove_empty_hashtag'].apply(lambda x: len(x))
df.head(50)

id  count  hate_speech  offensive_language  neither  positive  class  \
0   28842  3      0            0                   0        3         3       
1   5684   3      1            2                   0        0         1       
2   22263  3      1            2                   0        0         1       
3   40774  3      0            0                   0        3         3       
4   7082   3      0            0                   3        0         2       
5   33371  3      0            0                   0        3         3       
6   27911  3      0            0                   0        3         3       
7   42015  3      0            0                   0        3         3       
8   6788   3      2            1                   0        0         0       
9   43002  3      0            0                   0        3         3       
10  38422  3      0            0                   0        3         3       
11  3478   3      1            2                   0        0         1       
12  40522  3      0            0                   0        3         3       
13  27249  3      0            0                   0        3         3       
14  44284  3      0            0                   0        3         3       
15  21761  3      0            3                   0        0         1       
16  1209   3      1            2                   0        0         1       
17  44586  3      0            0                   0        3         3       
18  5691   3      0            1                   2        0         2       
19  4058   3      0            3                   0        0         1       
20  23806  3      0            3                   0        0         1       
21  18743  3      0            3                   0        0         1       
22  11243  3      0            3                   0        0         1       
23  32488  3      0            0                   0        3         3       
24  24328  6      0            6                   0        0         1       
25  11111  3      0            3                   0        0         1       
26  42792  3      0            0                   0        3         3       
27  22508  3      0            3                   0        0         1       
28  42811  3      0            0                   0        3         3       
29  11241  3      0            0                   3        0         2       
30  1962   3      0            3                   0        0         1       
31  7276   3      1            2                   0        0         1       
32  16881  3      1            2                   0        0         1       
33  9659   6      0            5                   1        0         1       
34  29656  3      0            0                   0        3         3       
35  42758  3      0            0                   0        3         3       
36  29254  3      0            0                   0        3         3       
37  16237  3      0            3                   0        0         1       
38  2471   3      1            2                   0        0         1       
39  46815  3      0            0                   0        3         3       
40  15448  3      0            2                   1        0         1       
41  11628  3      0            3                   0        0         1       
42  13762  3      0            3                   0        0         1       
43  17806  3      0            3                   0        0         1       
44  8719   3      0            3                   0        0         1       
45  18212  3      0            3                   0        0         1       
46  28663  3      0            0                   0        3         3       
47  22521  3      0            3                   0        0         1       
48  7762   3      2            1                   0        0         0       
49  41804  3      0            0                   0     

In [88]:
# Check that there are no tweets that are greater than 280 characters
dfLen = df.loc[df['tweet_length'] > 280]

In [89]:
dfLen.sort_values(by=['tweet_length'], ascending = False)

id  count  hate_speech  offensive_language  neither  positive  \
29982  18267  3      0            3                   0        0          
28044  13733  3      0            3                   0        0          
28352  1631   3      0            3                   0        0          
23454  7644   3      0            2                   1        0          
6865   19305  3      0            3                   0        0          
39283  19214  3      0            3                   0        0          
24023  16918  3      0            0                   3        0          

       class  \
29982  1       
28044  1       
28352  1       
23454  1       
6865   1       
39283  1       
24023  2       

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      tweet  \
29982  RT @TrxllLegend: One good girl is worth a thousand bitches\n\n&#128112; = &#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#8230;   
28044  No summer school? &#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515; eat a dick school. Im done with your bitch ass !!!!!!                                                                                                      
28352  &#8220;@Untouchable_T: Never seen so many perfect bitches til I made a Twitter &#128564; but &#128056;&#9749;&#65039;&#8221;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;&#128175;                                                                                                                          
23454  A guy on True Blood is getting his penis inspected and the doctor told him it look like an eggplant\n\n&#127814;&#127814;&#127814;&#127814;&#127814;&#127814;&#127814;&#127814;&#127814;&#127814;&#127814;&#1

In [90]:
# Remove spaces in tweets to count only characters
df['nospaces'] = df['remove_empty_hashtag'].str.replace(" ", "")

In [91]:
df['character_count'] = df['nospaces'].apply(lambda x: len(x))
df.head(50)

id  count  hate_speech  offensive_language  neither  positive  class  \
0   28842  3      0            0                   0        3         3       
1   5684   3      1            2                   0        0         1       
2   22263  3      1            2                   0        0         1       
3   40774  3      0            0                   0        3         3       
4   7082   3      0            0                   3        0         2       
5   33371  3      0            0                   0        3         3       
6   27911  3      0            0                   0        3         3       
7   42015  3      0            0                   0        3         3       
8   6788   3      2            1                   0        0         0       
9   43002  3      0            0                   0        3         3       
10  38422  3      0            0                   0        3         3       
11  3478   3      1            2                   0        0         1       
12  40522  3      0            0                   0        3         3       
13  27249  3      0            0                   0        3         3       
14  44284  3      0            0                   0        3         3       
15  21761  3      0            3                   0        0         1       
16  1209   3      1            2                   0        0         1       
17  44586  3      0            0                   0        3         3       
18  5691   3      0            1                   2        0         2       
19  4058   3      0            3                   0        0         1       
20  23806  3      0            3                   0        0         1       
21  18743  3      0            3                   0        0         1       
22  11243  3      0            3                   0        0         1       
23  32488  3      0            0                   0        3         3       
24  24328  6      0            6                   0        0         1       
25  11111  3      0            3                   0        0         1       
26  42792  3      0            0                   0        3         3       
27  22508  3      0            3                   0        0         1       
28  42811  3      0            0                   0        3         3       
29  11241  3      0            0                   3        0         2       
30  1962   3      0            3                   0        0         1       
31  7276   3      1            2                   0        0         1       
32  16881  3      1            2                   0        0         1       
33  9659   6      0            5                   1        0         1       
34  29656  3      0            0                   0        3         3       
35  42758  3      0            0                   0        3         3       
36  29254  3      0            0                   0        3         3       
37  16237  3      0            3                   0        0         1       
38  2471   3      1            2                   0        0         1       
39  46815  3      0            0                   0        3         3       
40  15448  3      0            2                   1        0         1       
41  11628  3      0            3                   0        0         1       
42  13762  3      0            3                   0        0         1       
43  17806  3      0            3                   0        0         1       
44  8719   3      0            3                   0        0         1       
45  18212  3      0            3                   0        0         1       
46  28663  3      0            0                   0        3         3       
47  22521  3      0            3                   0        0         1       
48  7762   3      2            1                   0        0         0       
49  41804  3      0            0                   0     

In [92]:
df.isna().sum()

id                      0
count                   0
hate_speech             0
offensive_language      0
neither                 0
positive                0
class                   0
tweet                   0
tweet_low               0
no_url                  0
handle_count            0
no_handle               0
no_special              0
remove_empty_hashtag    0
tweet_length            0
nospaces                0
character_count         0
dtype: int64

# Lemmatization with Parts of Speech

In [145]:
df['lemmatized'] = df['no_special'].apply(lambda x: getLemma(x))

In [146]:
df.head(10)

id  count  hate_speech  offensive_language  neither  positive  class  \
0  28842  3      0            0                   0        3         3       
1  5684   3      1            2                   0        0         1       
2  22263  3      1            2                   0        0         1       
3  40774  3      0            0                   0        3         3       
4  7082   3      0            0                   3        0         2       
5  33371  3      0            0                   0        3         3       
6  27911  3      0            0                   0        3         3       
7  42015  3      0            0                   0        3         3       
8  6788   3      2            1                   0        0         0       
9  43002  3      0            0                   0        3         3       

                                                                                                                               tweet  \
0  13 days to go #gettingthere                                                                                                         
1  @anggxo get off my twitter fag                                                                                                      
2  These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3  a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.     
4  @ramaxe1965 dubya hates todays teabagger movement.                                                                                  
5  i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness?   @user          
6  getting   for this weekends shows! #country #music #lylepierce                                                                      
7   @user my final legislative session day has officially begun!   @user @user #albany                                                 
8  @lucas_wright955 @MichaelGT03 faggots                                                                                               
9  a #bikini kind of life ð´ summer #palmtrees #breeze   #place #cali #california   #swimwearâ¦                                     

                                                                                                                           tweet_low  \
0  13 days to go #gettingthere                                                                                                         
1  @anggxo get off my twitter fag                                                                                                      
2  these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3  a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.     
4  @ramaxe1965 dubya hates todays teabagger movement.                                                                                  
5  i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness?   @user          
6  getting   for this weekends shows! #country #music #lylepierce                                                                      
7   @user my final legislative session day has officially begun!   @user @user #albany                                                 
8  @lucas_wright955 @michaelgt03 faggots                                                                                               
9  a #bikini kind of life ð´ summer #palmtrees #breeze   #place #cali #california   #swimwearâ¦                                     

                                                                                                                              no_url  \
0  13 days to go 

In [147]:
df['lemma1'] = df['lemmatized'].str.replace('# ', '#')
df.head(5)

id  count  hate_speech  offensive_language  neither  positive  class  \
0  28842  3      0            0                   0        3         3       
1  5684   3      1            2                   0        0         1       
2  22263  3      1            2                   0        0         1       
3  40774  3      0            0                   0        3         3       
4  7082   3      0            0                   3        0         2       

                                                                                                                               tweet  \
0  13 days to go #gettingthere                                                                                                         
1  @anggxo get off my twitter fag                                                                                                      
2  These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3  a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.     
4  @ramaxe1965 dubya hates todays teabagger movement.                                                                                  

                                                                                                                           tweet_low  \
0  13 days to go #gettingthere                                                                                                         
1  @anggxo get off my twitter fag                                                                                                      
2  these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3  a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.     
4  @ramaxe1965 dubya hates todays teabagger movement.                                                                                  

                                                                                                                              no_url  \
0  13 days to go #gettingthere                                                                                                         
1  @anggxo get off my twitter fag                                                                                                      
2  these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3  a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.     
4  @ramaxe1965 dubya hates todays teabagger movement.                                                                                  

   handle_count  \
0  0              
1  1              
2  0              
3  1              
4  1              

                                                                                                                      no_handle  \
0  13 days to go #gettingthere                                                                                                    
1   get off my twitter fag                                                                                                        
2  these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                            
3  a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the  racists cuz she's not white.     
4   dubya hates todays teabagger movement.                                                                                        

                                                                                                                     no_special  \
0     days to go #gettingthere                       

In [148]:
df['lemma_final'] = df['lemmatized'].str.replace(" '", "'")
df.head(15)

id  count  hate_speech  offensive_language  neither  positive  class  \
0   28842  3      0            0                   0        3         3       
1   5684   3      1            2                   0        0         1       
2   22263  3      1            2                   0        0         1       
3   40774  3      0            0                   0        3         3       
4   7082   3      0            0                   3        0         2       
5   33371  3      0            0                   0        3         3       
6   27911  3      0            0                   0        3         3       
7   42015  3      0            0                   0        3         3       
8   6788   3      2            1                   0        0         0       
9   43002  3      0            0                   0        3         3       
10  38422  3      0            0                   0        3         3       
11  3478   3      1            2                   0        0         1       
12  40522  3      0            0                   0        3         3       
13  27249  3      0            0                   0        3         3       
14  44284  3      0            0                   0        3         3       

                                                                                                                                tweet  \
0   13 days to go #gettingthere                                                                                                         
1   @anggxo get off my twitter fag                                                                                                      
2   These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3   a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.     
4   @ramaxe1965 dubya hates todays teabagger movement.                                                                                  
5   i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness?   @user          
6   getting   for this weekends shows! #country #music #lylepierce                                                                      
7    @user my final legislative session day has officially begun!   @user @user #albany                                                 
8   @lucas_wright955 @MichaelGT03 faggots                                                                                               
9   a #bikini kind of life ð´ summer #palmtrees #breeze   #place #cali #california   #swimwearâ¦                                     
10  very sad people with very sad lives, gossiping about people who are fuck all to do with them   #getalife                            
11  @HopOnTheBeast i found her right here, good job unfollowing me btw faggot http://t.co/kD3iSeIoLd                                    
12  missy elliot for marc jacobs is supa dupa fly and i'm   ab it                                                                       
13  working on something ð.                                                                                                          
14  this week like....                                                                                                                  

                                                                                                                            tweet_low  \
0   13 days to go #gettingthere                                                                                                         
1   @anggxo get off my twitter fag                                                                                                      
2   these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3

# Remove StopWords and Words Less than Two Characters

In [149]:
# Import stopwords in English
stop = stopwords.words('english')
df['tweet_no_stopwords'] = df['lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [150]:
df.head(10)

id  count  hate_speech  offensive_language  neither  positive  class  \
0  28842  3      0            0                   0        3         3       
1  5684   3      1            2                   0        0         1       
2  22263  3      1            2                   0        0         1       
3  40774  3      0            0                   0        3         3       
4  7082   3      0            0                   3        0         2       
5  33371  3      0            0                   0        3         3       
6  27911  3      0            0                   0        3         3       
7  42015  3      0            0                   0        3         3       
8  6788   3      2            1                   0        0         0       
9  43002  3      0            0                   0        3         3       

                                                                                                                               tweet  \
0  13 days to go #gettingthere                                                                                                         
1  @anggxo get off my twitter fag                                                                                                      
2  These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3  a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.     
4  @ramaxe1965 dubya hates todays teabagger movement.                                                                                  
5  i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness?   @user          
6  getting   for this weekends shows! #country #music #lylepierce                                                                      
7   @user my final legislative session day has officially begun!   @user @user #albany                                                 
8  @lucas_wright955 @MichaelGT03 faggots                                                                                               
9  a #bikini kind of life ð´ summer #palmtrees #breeze   #place #cali #california   #swimwearâ¦                                     

                                                                                                                           tweet_low  \
0  13 days to go #gettingthere                                                                                                         
1  @anggxo get off my twitter fag                                                                                                      
2  these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3  a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.     
4  @ramaxe1965 dubya hates todays teabagger movement.                                                                                  
5  i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness?   @user          
6  getting   for this weekends shows! #country #music #lylepierce                                                                      
7   @user my final legislative session day has officially begun!   @user @user #albany                                                 
8  @lucas_wright955 @michaelgt03 faggots                                                                                               
9  a #bikini kind of life ð´ summer #palmtrees #breeze   #place #cali #california   #swimwearâ¦                                     

                                                                                                                              no_url  \
0  13 days to go 

In [151]:
# Removing words shorter than two characters because they will likely not be relevant
df['tweet_no_stopwords_no_short'] = df['tweet_no_stopwords'].apply(lambda x: ' '.join([word for word in x.split() if
                                                                                      len(word) > 2]))
df.head(5)

id  count  hate_speech  offensive_language  neither  positive  class  \
0  28842  3      0            0                   0        3         3       
1  5684   3      1            2                   0        0         1       
2  22263  3      1            2                   0        0         1       
3  40774  3      0            0                   0        3         3       
4  7082   3      0            0                   3        0         2       

                                                                                                                               tweet  \
0  13 days to go #gettingthere                                                                                                         
1  @anggxo get off my twitter fag                                                                                                      
2  These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3  a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.     
4  @ramaxe1965 dubya hates todays teabagger movement.                                                                                  

                                                                                                                           tweet_low  \
0  13 days to go #gettingthere                                                                                                         
1  @anggxo get off my twitter fag                                                                                                      
2  these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3  a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.     
4  @ramaxe1965 dubya hates todays teabagger movement.                                                                                  

                                                                                                                              no_url  \
0  13 days to go #gettingthere                                                                                                         
1  @anggxo get off my twitter fag                                                                                                      
2  these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3  a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.     
4  @ramaxe1965 dubya hates todays teabagger movement.                                                                                  

   handle_count  \
0  0              
1  1              
2  0              
3  1              
4  1              

                                                                                                                      no_handle  \
0  13 days to go #gettingthere                                                                                                    
1   get off my twitter fag                                                                                                        
2  these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                            
3  a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the  racists cuz she's not white.     
4   dubya hates todays teabagger movement.                                                                                        

                                                                                                                     no_special  \
0     days to go #gettingthere                       

# Labels for Binary Classification

In [152]:
df['positive'].value_counts()

0    24773
3    21421
Name: positive, dtype: int64

In [153]:
# Exclude neutral tweets because we are building a binary model right now
df = df.loc[df['class'] != 2]

In [154]:
len(df)

42031

In [155]:
df['neg_label'] = df['class'].apply(lambda x: 0 if x == 3 else 1)

In [156]:
df['neg_label'].value_counts()

0    21421
1    20610
Name: neg_label, dtype: int64

# Dealing with Missing Values, Dropping Columns, Save for EDA

In [158]:
df.drop(['count', 'hate_speech', 'offensive_language', 'neither', 'positive', 'class', 'tweet', 'no_url', 'no_handle', 
        'no_special'], axis = 1, inplace = True)

In [160]:
df.drop(['id', 'nospaces'], axis = 1, inplace = True)

In [161]:
df.head(5)

tweet_low  \
0  13 days to go #gettingthere                                                                                                         
1  @anggxo get off my twitter fag                                                                                                      
2  these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.                                                                 
3  a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.     
5  i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness?   @user          

   handle_count  \
0  0              
1  1              
2  0              
3  1              
5  1              

                                                                                                           remove_empty_hashtag  \
0     days to go #gettingthere                                                                                                    
1   get off my twitter fag                                                                                                        
2  these hoes got more bodies than a cemetery                                                                                     
3  a friend just told me she's afraid to go to dc rally  amp  be attacked by #berniebros or the  racists cuz she's not white      
5  i've noticed a lot of #icontf   presentations mention happiness  wonder if profession has above average happiness              

   tweet_length  character_count  \
0  30            21                
1  23            18                
2  58            35                
3  124           95                
5  117           95                

                                                                                                               lemmatized  \
0  day to go # gettingthere                                                                                                 
1  get off my twitter fag                                                                                                   
2  these hoe get more body than a cemetery # # #                                                                            
3  a friend just tell me she 's afraid to go to dc rally amp be attack by # berniebros or the racist cuz she 's not white   
5  i 've notice a lot of # icontf presentation mention happiness wonder if profession have above average happiness          

                                                                                                                  lemma1  \
0  day to go #gettingthere                                                                                                 
1  get off my twitter fag                                                                                                  
2  these hoe get more body than a cemetery ###                                                                             
3  a friend just tell me she 's afraid to go to dc rally amp be attack by #berniebros or the racist cuz she 's not white   
5  i 've notice a lot of #icontf presentation mention happiness wonder if profession have above average happiness          

                                                                                                            lemma_final  \
0  day to go # gettingthere                                                                                               
1  get off my twitter fag                                                                                                 
2  these hoe get more body than a cemetery # # #                                                                          
3  a friend just tell me she's afraid to go to dc rally amp be attack by # berniebros or the racist cuz she's not white   
5  i've notice a lot of # icontf presentation mention happiness wonder if pr

In [162]:
df.isna().sum()

tweet_low                      0
handle_count                   0
remove_empty_hashtag           0
tweet_length                   0
character_count                0
lemmatized                     0
lemma1                         0
lemma_final                    0
tweet_no_stopwords             0
tweet_no_stopwords_no_short    0
neg_label                      0
dtype: int64

In [163]:
len(df)

42031

In [ ]:
# Save to file
